In [ ]:
#import packages
import os
import sys
import cv2
import numpy as np
import argparse
import string
import pandas as pd
from scipy import stats

#import plantcv
from plantcv import plantcv as pcv

In [ ]:
#open image 
def open_image(folder_path, image_name, output_folder_path):
    """
    Open an image using PlantCV and return the image, path, and filename.

    Parameters:
        folder_path (str): Full path to the folder containing the image.
        image_name (str): Name of the image file.
        output_folder_path (str): Full path where the cprocessed images are stored.

    Returns:
        image, path (directory + image), filename.
    """
    # Construct the full path to the image
    image_path = os.path.join(folder_path, image_name)

    # Read the image using PlantCV
    img, _, _ = pcv.readimage(filename=image_path)

    #save the processed image to the output folder
    output_path = os.path.join(output_folder_path, "processed_" + image_name)
    pcv.print_image(img=img, filename=output_path)

    return img, image_path, image_name

In [ ]:
#open image 
def open_image(folder_path, image_name, output_folder_path):
    """
    Open an image using PlantCV and return the image, path, and filename.

    Parameters:
        folder_path (str): Full path to the folder containing the image.
        image_name (str): Name of the image file.
        output_folder_path (str): Full path where the cprocessed images are stored.

    Returns:
        image, path (directory + image), filename.
    """
    # Construct the full path to the image
    image_path = os.path.join(folder_path, image_name)

    # Read the image using PlantCV
    img, _, _ = pcv.readimage(filename=image_path)

    #save the processed image to the output folder
    output_path = os.path.join(output_folder_path, "processed_" + image_name)
    pcv.print_image(img=img, filename=output_path)

    return img, image_path, image_name

In [ ]:
def canopy_analysis(folder_path, data_name_out, output_folder_path):
    '''
    This function will run the full analysis of canopy images and output
    a csv file with the final data
    
    Parameters:
        folder_path (string): Full path to the folder containing images to analyze.
        
        data_name_out (string): The name of the final dataset output (without .csv in name)
        
        output_folder_path (string): The full path where the classified and cropped 
                                     images output
    Returns:
        tuple: A DataFrame with image name and canopy measurement data.
    '''
    pcv.params.debug = "none"
    
    # Create empty dataframe to store final data
    dataset_full = pd.DataFrame() 

    # Loop through each image in folder
    for image in os.listdir(folder_path):

        # Only open images
        if image.lower().endswith(('.png', '.jpg', '.jpeg', '.tif', '.tiff')):
        
            # Open image
            img, path, img_filename = open_image(folder_path, image, output_folder_path) 

            #extract image name
            name = os.path.splitext(image)[0]
            
            #Convert RGB to grayscale 
            #may need to change the channel depending on lighting and background
            grayscale_img = pcv.rgb2gray_lab(rgb_img=img, channel="b")
                
            # Thresholding
            auto_thresh = pcv.threshold.otsu(gray_img=grayscale_img, object_type="light")

            # Calculate traits: 
            shape_image = pcv.analyze.size(img=img, labeled_mask=auto_thresh, n_labels=1)

            area = pcv.outputs.observations['default1']['area']['value']
            width = pcv.outputs.observations['default1']['width']['value']

            # Create dataset for the current image
            data = pd.DataFrame({ 
                    'Plant_ID': name,
                    'Area': area,
                    'Height': height,
                    'Width': width},
                    index=[0])
            
            # Add data to the full dataset 
            dataset_full = pd.concat([dataset_full, data])

    # Save dataset as a csv
    csv_path = os.path.join(output_folder_path, data_name_out + '.csv')
    dataset_full.to_csv(csv_path, header=True, index=False)

    # Return the dataset
    return dataset_full

In [ ]:
#Input all output and file names here

#folder_path (string): Full path to the folder containing images to analyze.
#data_name_out (string): The name of the final dataset output (without .csv in name)
#output_folder_path (string): The full path where the classified and cropped images outputfolder_path = "path to images"
data_name_out = "name of output file (.csv)"
output_folder_path = "path to output folder" #must be separate from the input folder path 

#run full analysis 
canopy_analysis(folder_path, data_name_out, output_folder_path) 